In [1]:
import pandas as pd
import numpy as np
import math
import swifter

from model_functions import Processing, TennisModel
from glicko import Glicko
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Files
DATASET_FULL = 'dataset/dataset_full_2023-05-22.csv'
DATASET = 'dataset/dataset_2023-05-22.csv'
COUNTRY_CODES = 'dataset/ioc_iso.csv'
TOURNAMENTS = 'dataset/tournaments.csv'
PLAYERS = 'dataset/atp_players.csv'
NEW_DATASET = 'dataset/unused_datasets/atp_matches_2023.csv'

pd.set_option('mode.chained_assignment', None)

In [3]:
model = TennisModel(DATASET, DATASET_FULL, TOURNAMENTS, COUNTRY_CODES, PLAYERS)

Model accuracy: 0.6961396057557727


In [4]:
matches = model.add_matches(NEW_DATASET)

Processing: 100%|██████████| 243/243 [00:02<00:00, 114.87it/s]


In [5]:
matches

,p1_age,p2_age,winner,year,day_cos,day_sin,p1_glicko2_rating,p1_glicko2_rd,p1_glicko2_vol,p2_glicko2_rating,...,day_of_year,tourney_level,p1_name,p2_name,winner_name,p1_iso,p2_iso,2weeks_ago,semester_ago,year_ago
0,18.3,26.3,1,1993,0.999852,0.017213,1500.000000,350.000000,0.060000,1500.000000,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25.2,21.8,0,1993,0.999852,0.017213,1500.000000,350.000000,0.060000,1500.000000,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20.9,18.0,1,1993,0.999852,0.017213,1500.000000,350.000000,0.060000,1500.000000,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21.3,20.6,0,1993,0.999852,0.017213,1500.000000,350.000000,0.060000,1500.000000,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20.5,18.4,0,1993,0.999852,0.017213,1500.000000,350.000000,0.060000,1500.000000,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725425,25.6,26.4,0,2023,-0.973118,0.230306,2150.826357,65.573844,0.059450,1938.631444,...,169.0,A,Taylor Fritz,Bernabe Zapata Miralles,Taylor Fritz,US,ES,20230605.0,20221219.0,20220619.0
725426,27.0,21.2,1,2023,-0.973118,0.230306,1743.565232,63.836789,0.059429,2066.463865,...,169.0,A,Jan Choinski,Lorenzo Musetti,Lorenzo Musetti,GB,IT,20230605.0,20221219.0,20220619.0
725427,24.4,20.6,1,2023,-0.973118,0.230306,1969.818179,62.407311,0.059707,1897.140367,...,169.0,A,J J Wolf,Ben Shelton,Ben Shelton,US,US,20230605.0,20221219.0,20220619.0
725428,27.6,24.9,0,2023,-0.973118,0.230306,1675.472134,63.493472,0.059522,2013.013285,...,169.0,A,Ryan Peniston,Ugo Humbert,Ryan Peniston,GB,FR,20230605.0,20221219.0,20220619.0


In [13]:
matches[(matches['p1_id'] == 202195) | (matches['p2_id'] == 202195)][['p1_cwins', 'p1_closses','p2_cwins',  'p2_closses', 'winner', 'p1_id', 'p2_id']]

,p1_cwins,p1_closses,p2_cwins,p2_closses,winner,p1_id,p2_id
548335,0.0,0.0,0.0,3.0,1,202195,104187
550475,0.0,0.0,0.0,1.0,1,202229,202195
550490,1.0,0.0,1.0,0.0,1,202195,126091
583560,0.0,1.0,0.0,4.0,1,202195,104947
584949,0.0,2.0,0.0,1.0,1,202195,106266
...,...,...,...,...,...,...,...
723028,0.0,5.0,0.0,2.0,1,202195,104527
724101,0.0,1.0,0.0,6.0,0,105550,202195
725197,0.0,7.0,3.0,0.0,1,202195,124116
725334,0.0,5.0,0.0,8.0,0,144817,202195


In [6]:
model.df

,p1_age,p2_age,winner,p1_glicko2_rating,p1_glicko2_rd,p1_glicko2_vol,p2_glicko2_rating,p2_glicko2_rd,p2_glicko2_vol,p1_tour_wins_last_sem,...,p1_weeks_inactive,p2_weeks_inactive,p1_tourney_wins,p1_tourney_losses,p2_tourney_wins,p2_tourney_losses,p1_cwins,p2_cwins,p1_closses,p2_closses
0,18.3,26.3,1,1500.000000,350.000000,0.060000,1500.000000,350.000000,0.060000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25.2,21.8,0,1500.000000,350.000000,0.060000,1500.000000,350.000000,0.060000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20.9,18.0,1,1500.000000,350.000000,0.060000,1500.000000,350.000000,0.060000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21.3,20.6,0,1500.000000,350.000000,0.060000,1500.000000,350.000000,0.060000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20.5,18.4,0,1500.000000,350.000000,0.060000,1500.000000,350.000000,0.060000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725425,25.6,26.4,0,2150.826357,65.573844,0.059450,1938.631444,62.819603,0.059135,32.0,...,0.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0
725426,27.0,21.2,1,1743.565232,63.836789,0.059429,2066.463865,63.661689,0.059625,1.0,...,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
725427,24.4,20.6,1,1969.818179,62.407311,0.059707,1897.140367,63.011963,0.059901,14.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
725428,27.6,24.9,0,1675.472134,63.493472,0.059522,2013.013285,62.337742,0.059456,0.0,...,0.0,0.0,3.0,2.0,0.0,0.0,0.0,1.0,8.0,0.0


In [20]:
new_match = {
    'tourney_name' : 'Roland Garros',
    'surface' : 'Clay',
    'tourney_level' : 'G',
    'tourney_date' : 20230529,
    'p1_name' : 'Radu Albot',
    'p2_name' : 'Patrick Kypson',
    'best_of' : 5
}

In [21]:
print(model.get_bet(new_match, 1.39, 3))

Processing: 100%|██████████| 1/1 [00:00<00:00, 90.89it/s]


None


In [ ]:
+0.22

In [5]:
match = model.predict(new_match)

Processing: 100%|██████████| 1/1 [00:00<00:00, 83.31it/s]


In [61]:
match

array([[0.30744124, 0.69255876]])

In [6]:
print(model.get_bet(new_match, 4.69, 1.18))

Processing: 100%|██████████| 1/1 [00:00<00:00, 90.89it/s]


(1, 'Guido Pella')


In [4]:
req_cols = ['tourney_name', 'surface', 'tourney_level', 'tourney_date', 'winner_id', 'winner_name', 'winner_ioc', 'winner_age', \
            'loser_id', 'loser_name', 'loser_ioc', 'loser_age', 'best_of']
new_df = pd.read_csv(NEW_DATASET, usecols=req_cols, dtype={'tourney_level': str})

In [5]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1637 entries, 0 to 1636
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tourney_name   1637 non-null   object 
 1   surface        1637 non-null   object 
 2   tourney_level  1637 non-null   object 
 3   tourney_date   1637 non-null   int64  
 4   winner_id      1637 non-null   int64  
 5   winner_name    1637 non-null   object 
 6   winner_ioc     1637 non-null   object 
 7   winner_age     1637 non-null   float64
 8   loser_id       1637 non-null   int64  
 9   loser_name     1637 non-null   object 
 10  loser_ioc      1637 non-null   object 
 11  loser_age      1631 non-null   float64
 12  best_of        1637 non-null   int64  
dtypes: float64(2), int64(4), object(7)
memory usage: 166.4+ KB


In [18]:
# Loading in new datasets
req_cols = ['tourney_name', 'surface', 'tourney_level', 'tourney_date', 'winner_id', 'winner_name', 'winner_ioc', 'winner_age', \
            'loser_id', 'loser_name', 'loser_ioc', 'loser_age', 'best_of']
new_df = pd.read_csv(NEW_DATASET, usecols=req_cols, dtype={'tourney_level': str})
new_df.index = range(new_df.shape[0])

# Read supporting datasets
df = pd.read_csv(DATASET, keep_default_na=False)
df.index = range(df.shape[0])
iso_df = pd.read_csv(COUNTRY_CODES, keep_default_na=False)
tournament_loc_df = pd.read_csv(TOURNAMENTS)
id_df = pd.read_csv(IDS, encoding = "ISO-8859-1")

# Initialize Processing
new_df_proc = Processing(new_df, tournament_loc_df, iso_df)
glicko = Glicko()
pd.set_option('mode.chained_assignment', None)

In [4]:
# Update the dataset
new_df = new_df_proc.prepare_append()
new_df = new_df_proc.append_onto(df)

missing_tourneys = new_df_proc.check_tourney_missing()
if len(missing_tourneys) == 0:
    new_df = new_df_proc.prepare_rolling_features()
else:
    print(missing_tourneys)

Processing: 100%|██████████| 243/243 [00:01<00:00, 124.84it/s]


In [5]:
# test = new_df[((new_df['p1_id'] == 126207) | (new_df['p2_id'] == 126207))]
test = new_df[(new_df['new_col'] == 1) & ((new_df['p1_id'] == 126207) | (new_df['p2_id'] == 126207))]
test[['p1_id','p2_id', 'winner', 'p1_cwins', 'p1_closses', 'p2_cwins', 'p2_closses']]

,p1_id,p2_id,winner,p1_cwins,p1_closses,p2_cwins,p2_closses
725242,126207,105936,0,1.0,0.0,0.0,2.0
725278,106234,126207,1,6.0,0.0,2.0,0.0
725296,100644,126207,0,4.0,0.0,3.0,0.0
725341,105526,126207,1,0.0,1.0,0.0,1.0
725343,126207,105916,0,1.0,0.0,0.0,1.0
725346,126207,207518,0,2.0,0.0,0.0,1.0
725353,208103,126207,1,0.0,1.0,3.0,0.0
725408,200624,126207,0,1.0,0.0,4.0,0.0
725418,122298,126207,1,0.0,3.0,0.0,1.0


In [ ]:
# Create helper features
df['2weeks_ago'] = df.apply(lambda row: processing.get_earlier_date(row['tourney_datetime'], '2 weeks'), axis=1)
df['semester_ago'] = df.apply(lambda row: processing.get_earlier_date(row['tourney_datetime'], 'semester'), axis=1)
df['year_ago'] = df.apply(lambda row: processing.get_earlier_date(row['tourney_datetime'], 'year'), axis=1)
df['2weeks_ago'] = df['2weeks_ago'].astype('int32')
df['semester_ago'] = df['semester_ago'].astype('int32')
df['year_ago'] = df['year_ago'].astype('int32')
processing.update_df(df)

In [ ]:
# Create Win Loss features (STEP 5)
df[['p1_tour_wins_last_sem', 'p1_tour_losses_last_sem']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 2, False, row['semester_ago']), axis=1)
df[['p1_tour_wins_last_year', 'p1_tour_losses_last_year']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 2, False, row['year_ago']), axis=1)
df[['p1_tour_wins_alltime', 'p1_tour_losses_alltime']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 2, True), axis=1)
df[['p1_qual/chal_wins_last_sem', 'p1_qual/chal_losses_last_sem']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 1, False, row['semester_ago']), axis=1)
df[['p1_qual/chal_wins_last_year', 'p1_qual/chal_losses_last_year']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 1, False, row['year_ago']), axis=1)
df[['p1_qual/chal_wins_alltime', 'p1_qual/chal_losses_alltime']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 1, True), axis=1)
df[['p1_futures_wins_last_sem', 'p1_futures_losses_last_sem']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 0, False, row['semester_ago']), axis=1)
df[['p1_futures_wins_last_year', 'p1_futures_losses_last_year']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 0, False, row['year_ago']), axis=1)
df[['p1_futures_wins_alltime', 'p1_futures_losses_alltime']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p1_id'], row.name, 0, True), axis=1)
df[['p2_tour_wins_last_sem', 'p2_tour_losses_last_sem']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 2, False, row['semester_ago']), axis=1)
df[['p2_tour_wins_last_year', 'p2_tour_losses_last_year']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 2, False, row['year_ago']), axis=1)
df[['p2_tour_wins_alltime', 'p2_tour_losses_alltime']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 2, True), axis=1)
df[['p2_qual/chal_wins_last_sem', 'p2_qual/chal_losses_last_sem']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 1, False, row['semester_ago']), axis=1)
df[['p2_qual/chal_wins_last_year', 'p2_qual/chal_losses_last_year']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 1, False, row['year_ago']), axis=1)
df[['p2_qual/chal_wins_alltime', 'p2_qual/chal_losses_alltime']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 1, True), axis=1)
df[['p2_futures_wins_last_sem', 'p2_futures_losses_last_sem']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 0, False, row['semester_ago']), axis=1)
df[['p2_futures_wins_last_year', 'p2_futures_losses_last_year']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 0, False, row['year_ago']), axis=1)
df[['p2_futures_wins_alltime', 'p2_futures_losses_alltime']] = df.swifter.apply(lambda row: processing.get_wins_and_losses(row['p2_id'], row.name, 0, True), axis=1)

In [ ]:
# Create Surface features (STEP 6)
df = pd.concat([df, pd.get_dummies(df['surface'], dtype=int)], axis=1)
df[['p1_wins_clay', 'p1_losses_clay']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p1_id'], 'Clay', row.name), axis=1)
df[['p1_wins_hard', 'p1_losses_hard']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p1_id'], 'Hard', row.name), axis=1)
df[['p1_wins_grass', 'p1_losses_grass']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p1_id'], 'Grass', row.name), axis=1)
df[['p1_wins_carpet', 'p1_losses_carpet']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p1_id'], 'Carpet', row.name), axis=1)
df[['p2_wins_clay', 'p2_losses_clay']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p2_id'], 'Clay', row.name), axis=1)
df[['p2_wins_hard', 'p2_losses_hard']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p2_id'], 'Hard', row.name), axis=1)
df[['p2_wins_grass', 'p2_losses_grass']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p2_id'], 'Grass', row.name), axis=1)
df[['p2_wins_carpet', 'p2_losses_carpet']] = df.swifter.apply(lambda row: processing.get_surface_games(row['p2_id'], 'Carpet', row.name), axis=1)

In [ ]:
# Get home advantage (STEP 7)
df[['p1_home_adv', 'p2_home_adv']] = df.swifter.apply(lambda row: processing.get_home_adv(row['p1_iso'], row['p2_iso'], row['tourney_name'], row['year']), axis=1)

In [ ]:
# Get head-to-head results (STEP 8)
df[['p1_h2h_wins', 'p2_h2h_wins']] = df.swifter.apply(lambda row: processing.get_h2h_wins(row['p1_id'], row['p2_id'], row.name), axis=1) 

In [ ]:
# Get Tourney W/L (STEP 9)
df['tourney_name_mod'] = df['tourney_name'].apply(processing.parse_tourney)
processing.update_df(df)
df[['p1_tourney_wins', 'p1_tourney_losses']] = df.swifter.apply(lambda row: processing.get_tourney_games(row['p1_id'], row['tourney_name_mod'], row.name), axis=1)
df[['p2_tourney_wins', 'p2_tourney_losses']] = df.swifter.apply(lambda row: processing.get_tourney_games(row['p2_id'], row['tourney_name_mod'], row.name), axis=1)

In [ ]:
# Get l2w games (STEP 10)
df['p1_last2w_games'] = df.swifter.apply(lambda row: processing.get_player_last2w_count(row['p1_id'], row['2weeks_ago'], row.name), axis=1)
df['p2_last2w_games'] = df.swifter.apply(lambda row: processing.get_player_last2w_count(row['p2_id'], row['2weeks_ago'], row.name), axis=1)

In [ ]:
# Get Inactive Weeks (STEP 11)
df['p1_weeks_inactive'] = df.swifter.apply(lambda row: processing.get_player_inactivity(row['p1_id'], row['day_of_year'], row['year'], row.name), axis=1)
df['p2_weeks_inactive'] = df.swifter.apply(lambda row: processing.get_player_inactivity(row['p2_id'], row['day_of_year'], row['year'], row.name), axis=1)

In [ ]:
# Get consecutive W/L (STEP 12)
processing.get_player_consecutive(df)

In [4]:
# Remove additional columns (STEP 13)
columns_to_drop = ['tourney_date', 'tourney_level', 'p1_name', 'p2_name', 'p1_iso', 'p2_iso', 'winner_name', \
                    'tourney_level_cond', 'tourney_datetime', 'tourney_date_int', '2weeks_ago', 'semester_ago', \
                    'year_ago', 'p1_id', 'p2_id', 'winner_id', 'tourney_name', 'surface', 'Carpet.1', 'Carpet.2', \
                    'Carpet.3', 'Carpet.4', 'Clay.1', 'Clay.2', 'Clay.3', 'Clay.4', 'Hard.1', 'Hard.2', 'Hard.3', \
                    'Hard.4', 'Grass.1', 'Grass.2', 'Grass.3', 'Grass.4', 'tourney_name_mod', 'month', 'day', 'day_of_year']

df = df.drop(columns_to_drop, axis=1)

In [ ]:
# # Hyperparameter Tuning of LogisticRegression
# lr = LogisticRegression(max_iter=10000)

# lr_grid = {
#     'C' : np.logspace(0, 4, num=10),
# }

# log_search = RandomizedSearchCV(lr, lr_grid, n_iter=5, cv=10, scoring='accuracy', n_jobs=-1, random_state=1)
# lr_result = log_search.fit(X_train_scaled, y_train)
# print('Best Score: %s' % lr_result.best_score_)
# print('Best Hyperparameters: %s' % lr_result.best_params_)

In [ ]:
# Create a function to build the Keras model
# def create_model(units=256, dropout=0.45, learning_rate=0.0001):
#     model = Sequential()
#     model.add(Dense(units=units, activation='relu', input_dim=X_train_scaled.shape[1]))
#     model.add(Dropout(rate=dropout))
#     model.add(Dense(units=1, activation='sigmoid'))
#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # Create the KerasClassifier
# cnn = KerasClassifier(build_fn=create_model, verbose=0)

# # Define the hyperparameters grid for random search
# cnn_grid = {
#     'units': [128, 256, 512],  # Number of units/neurons in the hidden layer
#     'dropout': [0.4, 0.45, 0.5],
#     'batch_size': [89, 178, 356, 712, 1424, 2848, 5696],  # Example batch sizes
#     'epochs': list(range(20, 60)),
#     'learning_rate': np.arange(0.0001, 0.0005, 0.00005)  # Learning rate
# }

# # Perform random search
# cnn_search = RandomizedSearchCV(cnn, cnn_grid, n_iter=5, cv=3, scoring='accuracy', n_jobs=-1, random_state=1)
# cnn_result = cnn_search.fit(X_train_scaled, y_train)

# # Print the best hyperparameters and accuracy
# print("Best Hyperparameters: ", cnn_result.best_params_)
# print("Best Accuracy: ", cnn_result.best_score_)

In [7]:
df = new_df

In [22]:
# Data splitting/scaling
n_train = math.ceil(df.shape[0] * 0.8)
train_df = df[:n_train]
test_df = df[n_train:]
X_train = train_df.drop(['winner'], axis=1)
y_train = train_df['winner']
X_test = test_df.drop(['winner'], axis=1)
y_test = test_df['winner']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
# Hyperparameter Tuning of SGDCClassifier
sgd = SGDClassifier(loss='log_loss', max_iter=10000)

sgd_grid = {
    'alpha': np.logspace(-4, 0, num=10),
    'l1_ratio': np.linspace(0, 1, num=10),
}

sgd_search = RandomizedSearchCV(sgd, sgd_grid, n_iter=5, cv=10, scoring='accuracy', n_jobs=-1, random_state=1)
sgd_result = sgd_search.fit(X_train_scaled, y_train)
print('Best Score: %s' % sgd_result.best_score_)
print('Best Hyperparameters: %s' % sgd_result.best_params_)


Best Score: 0.6943945531328105
Best Hyperparameters: {'l1_ratio': 0.3333333333333333, 'alpha': 0.002154434690031882}


In [24]:
accuracy_score(sgd_result.predict(X_test_scaled), y_test)

0.6953811785958066

In [62]:
# Get soft predictions on the test data
soft_predictions = sgd_result.best_estimator_.predict_proba(X_test_scaled)

# Store the soft predictions for the positive class
positive_probabilities = soft_predictions[:, 1]  # Assuming positive class is at index 1

# Print the list of soft predictions
print(positive_probabilities)

[0.4255136  0.73512268 0.53041751 ... 0.58575993 0.52538761 0.83153954]


In [64]:
sgd_result.predict(X_test_scaled)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [14]:
VAR = 0.1

def calculate_bet_sizing(model_prob, bookie_prob):
    sizing = calculate_kelly(model_prob, bookie_prob)
    return round(sizing * 10) if sizing > 0 else None

def calculate_kelly(model_prob, bookie_prob):
    if model_prob > 1 or bookie_prob > 1:
        raise Exception('Probability cannot be greater than 1')
    odds = 1 / bookie_prob
    top = (model_prob * odds - 1) ** 3
    bot = (odds - 1) * (((model_prob * odds - 1) ** 2) + ((odds ** 2) * (VAR ** 2)))

    return top / bot

def get_implied_probability(odds, format, is_minus=True):
    if format == 'American':
        return odds / (odds + 100) if is_minus else 100 / (odds + 100)
    elif format == 'Decimal':
        return 1 / odds
    else:
        return None

In [16]:
prob = get_implied_probability(1.02, 'Decimal')
calculate_bet_sizing(0.73, prob)